In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [3]:
from sklearn.cross_validation import train_test_split

In [4]:
from PIL import Image

In [5]:
import os

In [6]:
os.environ['THEANO_FLAGS'] = "device=gpu1"  
import theano

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
ERROR:theano.sandbox.cuda:nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import RMSprop, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
#from keras.optimizers import Adagrad
#from keras.optimizers import Adadelta


Using Theano backend.
/Users/swathisivadas/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [8]:
import time
import cv2

### Create a load function to load in data

In [9]:
def load_data(): 
    data = np.empty((690,3,300,300),dtype="float32")
    label = np.empty((690,),dtype ="uint8")
    imgs = os.listdir("images/train")
    num = len(imgs)
    j=0
    for i in range(num):
        if not imgs[i].startswith('.'):
            
            img = Image.open("images/train/"+imgs[i])
            try:
                arr = np.asarray (img, dtype ="float32")
                if len(arr.shape)>=3:
                    data [j,:,:,:] = [arr[:,:,0],arr[:,:, 1],arr[:,:, 2]] 
                    
                    if imgs[i].split('_')[0]=='burger':
                        label[j]=0
                    elif imgs[i].split('_')[0]=='fries':
                        label[j]=1
                    elif imgs[i].split('_')[0]=='spaghetti':
                        label[j]=2
                    j=j+1
            except Exception, e:
                print e
                continue
    return np.array(data), label

In [10]:
data,label = load_data()

float() argument must be a string or a number
could not broadcast input array from shape (250,250) into shape (300,300)
could not broadcast input array from shape (200,200) into shape (300,300)
could not broadcast input array from shape (200,200) into shape (300,300)
could not broadcast input array from shape (250,250) into shape (300,300)
could not broadcast input array from shape (163,220) into shape (300,300)
could not broadcast input array from shape (299,228) into shape (300,300)
could not broadcast input array from shape (250,250) into shape (300,300)
could not broadcast input array from shape (250,250) into shape (300,300)
could not broadcast input array from shape (170,136) into shape (300,300)


In [35]:
l = pd.DataFrame(label)

In [40]:
labels = pd.get_dummies(label)

In [43]:
labels = labels.astype(int)

In [11]:
data.shape

/Users/swathisivadas/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/swathisivadas/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/swathisivadas/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/swathisivadas/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/swathisivadas/anaconda/

(690, 3, 300, 300)

### Splitting data into train and test data

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data, labels)

In [45]:
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

### Setting up basic convolutional network

In [46]:
model = Sequential()
 
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(3, 300, 300)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
# Note: Keras does automatic shape inference.
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

In [47]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop())

In [48]:
start = time.time()
model.fit(X_train,y_train, batch_size=32, nb_epoch=1,verbose=1)
end = time.time()

Epoch 1/1
517/517 [==============================] - 1130s - loss: 11.9858   


In [17]:
X_train.shape

(517, 3, 300, 300)

In [50]:
model.evaluate(X_test,y_test,show_accuracy=True)

173/173 [==============================] - 77s     


[11.739190774156867, 0.27167630057803466]

In [51]:
end-start

1130.6147978305817

### Notes and next steps

1) Need to normalize the image

2) Setup on AWS to run more data

3) Increase the number of epochs

4) Need to increase number of epochs